<a href="https://colab.research.google.com/github/jenieto/computer-vision/blob/normalize-scikit-learn/computer-vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabajo M0

In [0]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive/')
!unzip -o "/content/drive/My Drive/Datasets/computer-vision-M2.zip" -d /content/

In [0]:
# Importamos librerias
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
import re
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [0]:
# Definimos variables
csv_path = '/content/computer-vision-M2/anotaciones_itemsEvaluables_v3.csv'
images_path = '/content/computer-vision-M2/dataset1'
figure = 'a1'
image_shape = (512, 512, 1)
test_dataset_size = 0.25

In [0]:
# Funciones para leer los datos
def read_csv_data():
  data = pd.read_csv(csv_path, sep=';', names=['dir', 'image', 'figure', 'coords'], index_col=False, skipinitialspace=True)
  for key in data.keys():
    data[key] = data[key].apply(str.replace, args=(' ', ''))
  return data

In [5]:
# Leemos los datos
data = read_csv_data()
data

,dir,image,figure,coords
0,REY_roi_manualSelection1,ROI_0_Psic_023Ev4.pdf_pg-26.jpg,a9,"(209,334),(194,380)"
1,REY_roi_manualSelection1,ROI_0_Psic_211Ev1.pdf_pg-27.jpg,a9,"(86,288),(86,338)"
2,REY_roi_manualSelection1,ROI_0_Psic_237Ev4.pdf_pg-24.jpg,a9,"(187,126),(195,18),(302,124),(185,126)"
3,REY_roi_manualSelection1,ROI_0_Psic_526Ev3.pdf_pg-23.jpg,a9,"(156,107),(152,62),(182,80),(260,106),(154,108)"
4,REY_roi_manualSelection1,ROI_0_Psic_440Ev2.pdf_pg-22.jpg,a9,"(184,84),(193,22),(219,34),(246,50),(307,80),(..."
...,...,...,...,...
5418,REY_roi_manualSelection1,ROI_0_Psic_517Ev3.pdf_pg-26.jpg,a1,"(35,50)"
5419,REY_roi_manualSelection1,ROI_0_Psic_408Ev2.pdf_pg-35.jpg,a1,"(29,34)"
5420,REY_roi_manualSelection1,ROI_0_Psic_210Ev5.pdf_pg-54.jpg,a1,"(33,39)"
5421,REY_roi_manualSelection1,ROI_0_Psic_514Ev3.pdf_pg-20.jpg,a1,"(34,33)"


In [0]:
# Inicializamos una intancia scaler
scaler = preprocessing.StandardScaler()

# Funcion para leer una imagen
def read_image_and_label(row, type=0, n_points=1):
  path = os.path.join(images_path, 'grafos_' + row['dir'], 'grafo_' + row['image'] + '.png')
  raw = cv2.imread(path,0)
  proc = None
  label = None
  if raw is not None: # TODO: algunas imagenes no existen
    
    shape = raw.shape

    raw1 = raw[:shape[0]//2, :shape[1]//2] # Imagen original
    raw2 = raw[:shape[0]//2, shape[1]//2:] # Imagen original invertida
    raw3 = raw[shape[0]//2:, :shape[1]//2] # Imagen de grafos detalle alto
    raw4 = raw[shape[0]//2:, shape[1]//2:] # Imagen de grafos detalle bajo

    raw1 = cv2.resize(raw1, dsize=(image_shape[0], image_shape[1])) # TODO: tendremos que redimensionar los puntos tambien
    raw2 = cv2.resize(raw2, dsize=(image_shape[0], image_shape[1]))
    raw3 = cv2.resize(raw3, dsize=(image_shape[0], image_shape[1]))
    raw4 = cv2.resize(raw4, dsize=(image_shape[0], image_shape[1]))
    
    proc1 = scaler.fit_transform(raw1)
    proc2 = scaler.fit_transform(raw2)
    proc3 = scaler.fit_transform(raw3)
    proc4 = scaler.fit_transform(raw4)

    if type == 0:
      raw = raw2 # Sólo se usa la imagen original invertida
      proc = proc2
    elif type == 1:
      raw = np.stack((raw2, raw3), axis=-1) # Se usa imagen original invertida + grafos alto detalle
      proc = np.stack((proc2, proc3), axis=-1) # Se usa imagen original invertida + grafos alto detalle
    elif type == 2:
      raw = np.stack((raw2, raw3, raw4), axis=-1) # Se usa imagen original invertida + grafos alto detalle + grafos bajo detalle
      proc = np.stack((proc2, proc3, proc4), axis=-1) # Se usa imagen original invertida + grafos alto detalle + grafos bajo detalle
    raw = np.expand_dims(raw, axis=0)
    proc = np.expand_dims(proc, axis=0)

    # Leer coordenadas
    flat = np.array([int(s) for s in re.findall('\d+', row['coords'])]) # Parsea los números en el texto
    mat = flat.reshape((-1, 2)) # Reagrupa las coordenadas en grupos de dos
    if mat.shape[0] >= n_points:
      label = (mat[:n_points, :] * np.array([image_shape[0] / shape[0], image_shape[1] / shape[1]])).flatten()
      label = np.expand_dims(label, axis=0)

  return raw_X, proc_X, raw_y, proc_y

# Funcion que devuelve las imagenes y las coordenadas
def read_images(data):
  firstIteration = True
  for _, row in data.iterrows():
    raw, proc, label = read_image_and_label(row, 2)
    if firstIteration == True:
      raws = raw
      X = proc
      y = label
      firstIteration = False
    elif raw is not None and label is not None:
      raws = np.concatenate((raws, raw), axis=0)
      X = np.concatenate((X, proc), axis=0)
      y = np.concatenate((y, label), axis=0)
  # raws = np.expand_dims(np.array(raws), axis=-1)
  # X = np.expand_dims(np.array(X), axis=-1)
  # y = np.array(y)
  return raws, X, y

In [34]:
# Generamos las imagenes y los outputs
raws, X, y = read_images(data[data['figure'] == 'a1'])
print(raws.shape)
print(X.shape)
print(y.shape)

ValueError: ignored

In [0]:
# Mostramos una imagen con el punto de la figura marcado
def show_image(image, point=None):
  if point is not None:
    cv2_imshow(cv2.circle(cv2.cvtColor(image[:, :, 0], cv2.COLOR_GRAY2BGR), (point[0], point[1]), 3, (0, 0, 255), -1))

show_image(raws[50], labels[50])

In [0]:
# Creamos el modelo
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(2, activation='relu'))
  return model

# Creamos los datasets de train y test
def generate_datasets(X, y):
  return train_test_split(X, y, test_size=test_dataset_size)

In [0]:
model = create_model()
train_images, test_images, train_labels, test_labels = generate_datasets(procs, labels)

In [0]:
# Compilamos y entrenamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)